In [1]:
import tensorflow as tf
import numpy as np

In [2]:
train_file = open("../../data/iyer/train.txt")

In [3]:
train_file_contents = train_file.readlines()

In [4]:
len(train_file_contents)

52997

In [5]:
train = []
for line in train_file_contents:
    items = line.split('\t')
    if len(items) == 5:
        train.append(line.split('\t')[3])

In [6]:
len(train)

52795

In [7]:
len(train[0])

40

In [8]:
len(train[1])

294

In [9]:
train = [ex for ex in train if len(ex) < 80]

In [10]:
train

['public string Name { get; private set; }',
 'using System.ComponentModel.DataAnnotations.Schema;',
 'var form = (Form)Activator.CreateInstance(myType);\\nform.ShowDialog();',
 'System.Reflection.Assembly.Load("assemblyname").GetName().Version.ToString()',
 'SwitchCell sc = (SwitchCell)tv.Root[0][0];\\nbool booleanprop = sc.On;',
 'Marshal.SizeOf(typeof(T))',
 '...\\nsq.Where(TableTwo.NameColumn).Like("A%");',
 'catch (Exception ex) {\\n    throw;\\n}',
 'Microsoft.VisualBasic.FileIO',
 'var result = _items.SelectMany (x => x.Get (something));',
 'if(row.Table == null)\\n{\\n}\\nelse\\n{\\n}',
 'r = (v < 0.01 && r > 0.0) ? 0.01 : v;',
 'IEnumerable<SomeClass>',
 'Regex r1 = new Regex(startTag + @"([A-Za-z0-9\\-]+)" + endTag);',
 'AppDomain.ProcessExit',
 'base(CloudConfigurationManager.GetSetting("YourEntities"))',
 "string[] parts = s.Split(',').Select(x => x.Trim()).ToArray();",
 'System.Data.DataTable',
 'Regex.Match("Meeting with developer 60min", @"(\\d+min)").Groups[1].ToString(

In [11]:
import sys
sys.path.append("../../src")
from text_data_utils import *
from os import path
import gensim

In [12]:
train_tok = tokenize_texts(train)

In [13]:
wv = gensim.models.Word2Vec(train_tok).wv

In [14]:
max_len = max(len(text) for text in train_tok)

In [15]:
train_tensor = tokenized_texts_to_tensor(train_tok, wv, max_len)

In [16]:
train_tensor[0, 0]

array([-0.29127407, -0.02719963, -0.09701791, -0.01904868,  0.37199003,
        0.03572746,  0.10522439, -0.07227133, -0.07921956, -0.13675906,
       -0.02102582,  0.07994274,  0.171011  , -0.55666226, -0.06683917,
        0.20384969,  0.06493412, -0.40339389,  0.01753006, -0.35861248,
       -0.36469406, -0.09782506, -0.10046019,  0.44919062,  0.10548551,
       -0.22300653,  0.25525886,  0.47236371,  0.63321942, -0.0090921 ,
       -0.03166737, -0.13198887, -0.33233994, -0.19006875,  0.44807568,
       -0.34372023,  0.21132791,  0.11006206,  0.05137694, -0.42234969,
        0.05995608,  0.37893799,  0.18727832,  0.09161738,  0.23722912,
       -0.12036612, -0.26097977, -0.55280417, -0.19025552, -0.22830197,
       -0.13825363, -0.10609785,  0.04180995,  0.25652802,  0.20715816,
        0.04503926, -0.34815392,  0.13494204, -0.4226158 , -0.71093291,
       -0.06344077, -0.51190168, -0.34197372,  0.12502722,  0.2006547 ,
       -0.45547286, -0.63496774, -0.62932605,  0.61704397,  0.20

In [17]:
train_tensor_fl = np.reshape(train_tensor, (train_tensor.shape[0], train_tensor.shape[1] * train_tensor.shape[2]))

In [18]:
train_tensor.shape

(12921, 74, 100)

In [19]:
train_tensor_fl.shape

(12921, 7400)

In [20]:
val_file_contents = open('../../data/iyer/valid.txt').readlines()
val = []
for line in val_file_contents:
    items = line.split('\t')
    if len(items) == 5:
        sample = line.split('\t')[3]
        if len(sample) <= max_len:
            val.append(sample)
val_tok = tokenize_texts(val)
val_tensor = tokenized_texts_to_tensor(val_tok, wv, max_len)
val_tensor_fl = np.reshape(val_tensor, (val_tensor.shape[0], val_tensor.shape[1] * val_tensor.shape[2]))

In [21]:
from mlp_vae import MLPVariationalAutoEncoder

In [22]:
latent_dim = 256

In [23]:
model = MLPVariationalAutoEncoder(train_tensor_fl.shape[1], latent_dim, [2048, 1024], final_activation='linear')
model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(train_tensor_fl, train_tensor_fl, batch_size=512, epochs=25, verbose=1, shuffle=True,
                    validation_data=(val_tensor_fl, val_tensor_fl))

Train on 12921 samples, validate on 1573 samples
Epoch 1/25
12921/12921 [==============================] - 5s 400us/sample - loss: 0.1569 - val_loss: 0.0319
Epoch 2/25
12921/12921 [==============================] - 3s 270us/sample - loss: 0.0288 - val_loss: 0.0266
Epoch 3/25
12921/12921 [==============================] - 3s 270us/sample - loss: 0.0249 - val_loss: 0.0236
Epoch 4/25
12921/12921 [==============================] - 3s 270us/sample - loss: 0.0240 - val_loss: 0.0216
Epoch 5/25
12921/12921 [==============================] - 4s 272us/sample - loss: 0.0216 - val_loss: 0.0201
Epoch 6/25
12921/12921 [==============================] - 3s 271us/sample - loss: 0.0203 - val_loss: 0.0195
Epoch 7/25
12921/12921 [==============================] - 3s 270us/sample - loss: 0.0187 - val_loss: 0.0233
Epoch 8/25
12921/12921 [==============================] - 3s 269us/sample - loss: 0.0195 - val_loss: 0.0191
Epoch 9/25
12921/12921 [==============================] - 3s 269us/sample - loss: 0.016

In [24]:
import random

In [25]:
random.seed()
random_idx = random.randrange(train_tensor.shape[0])

In [26]:
print("(Training Set) Input: ", tensor_to_tokenized_texts(np.array([train_tensor[random_idx]]), wv)[0])

(Training Set) Input:  ['<s>', '<UNK>', '=', 'true', '</s>']


In [27]:
rec = np.reshape(model.predict(np.array([train_tensor_fl[random_idx]])), (1, train_tensor.shape[1], train_tensor.shape[2]))

In [28]:
print("(Training Set) Reconstructed: ", tensor_to_tokenized_texts(rec, wv)[0])

(Training Set) Reconstructed:  ['<s>', '00', '(', ')', '</s>']


In [29]:
wv.similar_by_vector(rec[0, 3])

[(')', 0.945659875869751),
 ('Amount', 0.9214717149734497),
 ('TypeName', 0.9140221476554871),
 ('ToCharArray', 0.9114137887954712),
 ('Last', 0.9075021743774414),
 ('GetLength', 0.9072589874267578),
 ('Length', 0.9068576693534851),
 ('Name', 0.9003871083259583),
 ('000', 0.8972765803337097),
 ('rotation', 0.8948014378547668)]

In [30]:
test_file_contents = open('../../data/iyer/test.txt').readlines()
test = []
for line in test_file_contents:
    items = line.split('\t')
    if len(items) == 5:
        sample = line.split('\t')[3]
        if len(sample) < max_len:
            test.append(sample)
test_tok = tokenize_texts(test)
test_tensor = tokenized_texts_to_tensor(test_tok, wv, max_len)
test_tensor_fl = np.reshape(test_tensor, (test_tensor.shape[0], test_tensor.shape[1] * test_tensor.shape[2]))

In [31]:
model.evaluate(test_tensor_fl, test_tensor_fl, verbose=0)

0.039439421686493815

In [32]:
random.seed()
random_idx = random.randrange(test_tensor.shape[0])

In [33]:
print("(Test Set) Input: ", tensor_to_tokenized_texts(np.array([test_tensor[random_idx]]), wv)[0])

(Test Set) Input:  ['<s>', '<UNK>', '.', '<UNK>', '=', '<UNK>', '.', '<UNK>', ';', '</s>']


In [34]:
print("(Test Set) Reconstructed: ", tensor_to_tokenized_texts(np.reshape(model.predict(np.array([test_tensor_fl[random_idx]])), (1, test_tensor.shape[1], test_tensor.shape[2])), wv)[0])

(Test Set) Reconstructed:  ['<s>', '<UNK>', '.', 'public', '=', 'UTF8', '.', 'Application', ';', '</s>']
